In [84]:
import numpy as np
import os
import matplotlib
import pandas as pd

In [85]:
#read in whatever datafile you want
def read_datafunc(datafile):
    read_data = pd.read_csv(datafile)
    return read_data

In [86]:
#summarize the size and columns, of the data
def summarize_data(datafile):
    read_data = read_datafunc(datafile)
    print(datafile)
    print('Dimensions:', read_data.shape)
    print('Columns:', list(read_data.columns))
    print('Indicators:', list(read_data.nomindicador.unique()))
    return read_data
    

df = summarize_data('../data/Indicadores_de_Salud.csv')

../data/Indicadores_de_Salud.csv
Dimensions: (2868, 102)
Columns: ['consecutivo', 'idindicador', 'nomindicador', 'idunidad', 'nomunidad', 'iddepto', 'nomdepto', 'idmpio', 'nommpio', 'idregion', 'nomregion', 'idsubregion', 'nomsubregion', 'idregimen', 'nomregimen', 'ideps', 'nomeps', 'idips', 'nomips', 'idzona', 'nomzona', 'ideducacion', 'nomeducacion', 'idocupacion', 'nomocupacion', 'idedad', 'nomedad', 'idestadocivil', 'nomestadocivil', 'idsexo', 'nomsexo', 'idetnia', 'nometnia', 'yea1990', 'yea1991', 'yea1992', 'yea1993', 'yea1994', 'yea1995', 'yea1996', 'yea1997', 'yea1998', 'yea1999', 'yea2000', 'yea2001', 'yea2002', 'yea2003', 'yea2004', 'yea2005', 'yea2006', 'yea2007', 'yea2008', 'yea2009', 'yea2010', 'yea2011', 'yea2012', 'fue1990', 'fue1991', 'fue1992', 'fue1993', 'fue1994', 'fue1995', 'fue1996', 'fue1997', 'fue1998', 'fue1999', 'fue2000', 'fue2001', 'fue2002', 'fue2003', 'fue2004', 'fue2005', 'fue2006', 'fue2007', 'fue2008', 'fue2009', 'fue2010', 'fue2011', 'fue2012', 'obs1990

In [87]:
#removing columns that have highly sparse data
sparsity_threshold = .5
df_nonsparse_columns = df.loc[:,(df.isnull().sum()/len(df)) < sparsity_threshold]

#look at the new shape of the dataset
print(df_nonsparse_columns.shape)
print(df_nonsparse_columns.columns)

(2868, 21)
Index(['consecutivo', 'idindicador', 'nomindicador', 'idunidad', 'nomunidad',
       'iddepto', 'nomdepto', 'idmpio', 'nommpio', 'yea2005', 'yea2006',
       'yea2007', 'yea2008', 'yea2009', 'yea2010', 'fue2005', 'fue2006',
       'fue2007', 'fue2008', 'fue2009', 'fue2010'],
      dtype='object')


In [88]:
#looking at new indicadors
print(df_nonsparse_columns[['idindicador', 'nomindicador']].drop_duplicates())
print(len(df_nonsparse_columns.idindicador.unique()))

    idindicador                                       nomindicador
0      ttdiesti               Tasa de mortalidad infantil estimada
1      ctrmormt                       Mortalidad materna a 42 días
4      caelacex  Duración de la lactancia materna exclusiva en ...
6      pcrparin                  Porcentaje partos institucionales
24     rtrmorpu              Razón de mortalidad materna a 42 días
36     pbepredc                Prevalencia de desnutrición crónica
42     ptrldeng                          Letalidad de Dengue grave
45     pbretaia  Porcentaje de brotes de ETA con identificación...
55     pcrperca  Porcentaje de partos atendidos por personal ca...
57     pbehtaco  Porcentaje de población con hipertension arter...
62     igrtgfec                          Tasa Global de Fecundidad
63     pbrmivih  Porcentaje de transmisión materno infantil de VIH
69     ttdnesti   Tasa de mortalidad estimada en menores de 5 años
71     rgrtgfec                         Tasa General de Fecund

In [119]:
##drop all rows where yea2005:yea2010 are all empty
not_null = df_nonsparse_columns.dropna(subset=['yea2005','yea2006','yea2007','yea2008','yea2009','yea2010'], 
                                       how='all', thresh=2)

In [120]:
## looking at the new indicadores we kept
print(not_null.shape)
print(df_nonsparse_columns.shape)
indi = not_null[['idindicador', 'nomindicador']].drop_duplicates()
print(indi, len(indi))

(2626, 21)
(2868, 21)
    idindicador                                       nomindicador
0      ttdiesti               Tasa de mortalidad infantil estimada
1      ctrmormt                       Mortalidad materna a 42 días
6      pcrparin                  Porcentaje partos institucionales
24     rtrmorpu              Razón de mortalidad materna a 42 días
36     pbepredc                Prevalencia de desnutrición crónica
42     ptrldeng                          Letalidad de Dengue grave
55     pcrperca  Porcentaje de partos atendidos por personal ca...
63     pbrmivih  Porcentaje de transmisión materno infantil de VIH
69     ttdnesti   Tasa de mortalidad estimada en menores de 5 años
96     pcrrncpn  Porcentaje de nacidos vivos con cuatro o más c...
103    cgrnacim                        Número anual de nacimientos
247    pgeembar  Porcentaje de mujeres de 15 a 19 años alguna v...
300    paemetmo  Porcentaje de mujeres con uso actual de método...
671    caelacex  Duración de la lactanci

In [121]:
##group the dataframe by departamento and indicador
by_depto_indicador = not_null.groupby(by=['iddepto', 'idindicador'])

In [122]:
#keeping only yea2005:yea2010 columns find the average of the value by departamento

mean = by_depto_indicador[['yea2005','yea2006','yea2007','yea2008','yea2009','yea2010']].agg('mean')
median = by_depto_indicador[['yea2005','yea2006','yea2007','yea2008','yea2009','yea2010']].agg('median')

In [123]:
##ungrouping and setting index to be columns
median_reset = median.reset_index(level=['iddepto', 'idindicador'])

#melting to get into one row to see which indicadores are actually very sparse
median_melted = pd.melt(median_reset, id_vars=['iddepto', 'idindicador'], var_name='year')
median_melted_not_NA = pd.melt(median_reset, id_vars=['iddepto', 'idindicador'], var_name='year').dropna()

print(len(median_melted), len(median_melted_not_NA))

2256 1898


In [124]:
by_indicador_melted = median_melted.groupby('idindicador')
by_indicador_melted_not_NA = median_melted_not_NA.groupby('idindicador')

In [125]:
print(by_indicador_melted['idindicador'].value_counts().sort_values())
print(by_indicador_melted_not_NA['idindicador'].value_counts().sort_values())

idindicador  idindicador
caelacex     caelacex         6
paemetmo     paemetmo         6
pbepredg     pbepredg         6
pbepredc     pbepredc        78
pbrmivih     pbrmivih       150
ttdnesti     ttdnesti       174
ptrldeng     ptrldeng       186
pcrparin     pcrparin       204
pcrperca     pcrperca       204
pcrrncpn     pcrrncpn       204
pgeembar     pgeembar       204
rtrmorpu     rtrmorpu       204
ttdiesti     ttdiesti       204
cgrnacim     cgrnacim       210
ctrmormt     ctrmormt       216
Name: idindicador, dtype: int64
idindicador  idindicador
caelacex     caelacex         2
paemetmo     paemetmo         2
pbepredg     pbepredg         2
pbepredc     pbepredc        26
pbrmivih     pbrmivih        56
pgeembar     pgeembar        68
ptrldeng     ptrldeng       122
ttdnesti     ttdnesti       174
pcrparin     pcrparin       204
pcrperca     pcrperca       204
pcrrncpn     pcrrncpn       204
rtrmorpu     rtrmorpu       204
ttdiesti     ttdiesti       204
cgrnacim     cgrnacim 

In [114]:
keep = by_indicador_melted_not_NA['idindicador'].value_counts().sort_values()>150
print(keep)

idindicador  idindicador
ptrldeng     ptrldeng       False
pcrparin     pcrparin        True
ttdiesti     ttdiesti        True
Name: idindicador, dtype: bool


In [115]:
by_indicador_not_NA.head()

,iddepto,idindicador,year,value
0,5.0,cgrnacim,yea2005,94636.000000
1,5.0,ctrmormt,yea2005,61.000000
2,5.0,pbepredc,yea2005,11.400000
4,5.0,pcrparin,yea2005,97.650000
5,5.0,pcrperca,yea2005,97.789425
6,5.0,pcrrncpn,yea2005,84.310000
7,5.0,pgeembar,yea2005,22.100000
9,5.0,rtrmorpu,yea2005,64.460000
10,5.0,ttdiesti,yea2005,18.315000
11,5.0,ttdnesti,yea2005,25.130432
